# Rotations

In [1]:
import os
import sys
sys.path.append('/home/ng213/code/sharpy')
import numpy as np
import sharpy.utils.algebra as algebra

In [2]:
i = np.array([1, 0, 0])
j = np.array([0, 1, 0])
k = np.array([0, 0, 1])

## Rotation matrices with quaternions

Understand quat to rotation matrices

In [3]:
euler_rot_angle = np.array([0,90,0])*np.pi/180

In [4]:
quat_new = algebra.euler2quat_ag(euler_rot_angle)
print(quat_new)

[0.70710678 0.         0.70710678 0.        ]


In [5]:
quat_old = algebra.euler2quat(euler_rot_angle)
print(quat_old)

[0.70710678 0.         0.70710678 0.        ]


In [6]:
Cga_new = algebra.quat2rotation(quat_new)
Cga_old = algebra.quat2rotation(quat_old)

In [7]:
aircraft_nose = np.array([1, 0, 0])

In [8]:
Cga_new.T.dot(aircraft_nose)

array([2.22044605e-16, 0.00000000e+00, 1.00000000e+00])

In [9]:
Cga_old.T.dot(aircraft_nose)

array([2.22044605e-16, 0.00000000e+00, 1.00000000e+00])

# From an Euler angle to a Quaternion

## Current Implementation 
As currently implemented in SHARPy, the routine goes as:
* `euler2quat()`
    - `Cag = euler2rot(euler)` 
        + `euler2rot` uses the three rotation matrices, where the y-axis one (pitch) is transposed
    - `quat = rotation2quat(Cag.T)`
    

In [10]:
euler = np.array([0, 90, 0]) * np.pi / 180
print(euler)

[0.         1.57079633 0.        ]


In [11]:
Cag = algebra.euler2rot(euler)
print(Cag)

[[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17]]


In [28]:
q = algebra.rotation2quat(Cag)
print(q)

[0.70710678 0.         0.70710678 0.        ]


## New implementation

* `euler2quat_ag()`
    - `Cag = euler2rot_ag(euler)` 
        + `euler2rot` uses the three rotation matrices, with the rotations consistent with a right hand, SEU frame
    - `quat = rotation2quat(Cag)`

In [13]:
print(euler)

[0.         1.57079633 0.        ]


In [14]:
Cag = algebra.euler2rotation_ag(euler)
print(Cag)

[[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17]]


In [15]:
q = algebra.rotation2quat(Cag)
print(q)

[0.70710678 0.         0.70710678 0.        ]


## Analytical Rotation Matrix

In [16]:
def Cag_analytical(euler):
    phi = euler[0]
    theta = euler[1]
    psi = euler[2]
    
    cp = np.cos(phi)
    sp = np.sin(phi)
    
    ct = np.cos(theta)
    st = np.sin(theta)
    
    cs = np.cos(psi)
    ss = np.sin(psi)
    
    Cag = np.zeros((3,3))
    
    Cag[0, 0] = ct*cs
    Cag[0, 1] = -ct*ss
    Cag[0, 2] = st
    
    Cag[1, 0] = cp*ss + sp*st*cs
    Cag[1, 1] = cp*cs - sp*st*ss
    Cag[1, 2] = -sp*ct
    
    Cag[2, 0] = sp*ss - cp*st*cs
    Cag[2, 1] = sp*cs + cp*st*ss
    Cag[2, 2] = cp*ct
    
    return Cag

In [17]:
def Cag_n(euler):
    phi = euler[0]
    theta = euler[1]
    psi = euler[2]
    
    cp = np.cos(phi)
    sp = np.sin(phi)
    
    ct = np.cos(theta)
    st = np.sin(theta)
    
    cs = np.cos(psi)
    ss = np.sin(psi)
    
    Cag = np.zeros((3,3))
    
    Cag = algebra.rotation3d_x(phi).dot(algebra.rotation3d_y_ag(theta).dot(algebra.rotation3d_z(psi)))
    
    return Cag

In [18]:
euler = np.array([20, 10, -5])*np.pi/180

Cag_n(euler) - Cag_analytical(euler)

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-6.9388939e-18,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00]])

In [19]:
r = -k

In [20]:
rot5 = np.array([0, 5, 0])*np.pi/180

In [21]:
Cag_n(rot5).dot(r)

array([-0.08715574,  0.        , -0.9961947 ])

In [22]:
algebra.euler2rotation_ag(rot5).dot(r)

array([-0.08715574,  0.        , -0.9961947 ])

In [23]:
algebra.euler2rot(rot5).dot(r)

array([-0.08715574,  0.        , -0.9961947 ])

## Propagation equations

In [24]:
def prop_euler_to_rot(euler):
    phi = euler[0]
    theta = euler[1]
    psi = euler[2]
    
    cp = np.cos(phi)
    sp = np.sin(phi)
    
    ct = np.cos(theta)
    st = np.sin(theta)
    
    cs = np.cos(psi)
    ss = np.sin(psi)
    
    RE = np.zeros((3,3))
    
    RE[0, 0] = 1
    RE[0, 2] = st
    RE[1, 1] = cp
    RE[1, 2] = -sp*ct
    RE[2, 1] = sp
    RE[2, 2] = cp*ct
    
    return RE

In [25]:
def prop_rot_to_euler(euler):
    phi = euler[0]
    theta = euler[1]
    psi = euler[2]
    
    cp = np.cos(phi)
    sp = np.sin(phi)
    
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
    
    cs = np.cos(psi)
    ss = np.sin(psi)
    
    RE = np.zeros((3,3))
    
    RE[0, 0] = 1
    RE[0, 1] = sp*tt
    RE[0, 2] = -cp*tt
    RE[1, 1] = cp
    RE[1, 2] = sp
    RE[2, 1] = -sp/ct
    RE[2, 2] = cp/ct
    
    return RE

In [26]:
euler

array([ 0.34906585,  0.17453293, -0.08726646])

In [27]:
prop_euler_to_rot(euler).dot(prop_rot_to_euler(euler))

array([[ 1.00000000e+00,  4.13057706e-19, -2.17329298e-18],
       [ 0.00000000e+00,  1.00000000e+00,  4.50889682e-19],
       [ 0.00000000e+00, -4.16257982e-18,  1.00000000e+00]])